In [1]:
from langchain.llms.azureml_endpoint import AzureMLModel

In [2]:
import os

In [3]:
azure_llm = AzureMLModel(
    endpoint_url=os.getenv("ENDPOINT_URL"),
    endpoint_api_key=os.getenv("ENDPOINT_API_KEY"),
    deployment_name="matthew-gpt-2",
    model_kwargs={"temperature": 0.9, "top_p": 0.3}
)

In [5]:
resp = azure_llm("Hi! How are you doing today?")
resp

"Hi! How are you doing today? I've noticed your mood changes, but not your actions. Maybe your mom just doesn't like to talk with you, which might've been the reason. Whatever the excuse was before, it's not helping you"

In [5]:
llm_result = azure_llm.generate(["Tell me a funny joke", "Tell me a sad poem"] * 10)

In [6]:
llm_result

LLMResult(generations=[[Generation(text='Tell me a funny joke and I\'ll eat it!"\n\nI\'m surprised at how many people on this crew seemed like they cared. I know that the people behind it were looking for it, but it seemed like they wanted people to at least', generation_info=None)], [Generation(text='Tell me a sad poem," I said to her, "a very sad poem." I went on saying, in a very sad tone, "The girl looks young, with a little beauty in her, and one of those good-looking girls in', generation_info=None)], [Generation(text='Tell me a funny joke and I\'ll do it!"\n\nAt this point, he pauses for a second, staring into her eyes. "Or maybe if you just ask, I\'ll do it to you," he says.\n\nAfter a', generation_info=None)], [Generation(text='Tell me a sad poem that says I can save you?"\n\n"Don\'t you have a wife or children?"\n\n"All I can do is help you get back on your feet," I said, but he wasn\'t satisfied.\n', generation_info=None)], [Generation(text='Tell me a funny joke or have an i

In [6]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [7]:
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "light", "antonym": "dark"},
    {"word": "fast", "antonym": "slow"},
    {"word": "high", "antonym": "low"},
    {"word": "windy", "antonym": "calm"},
    {"word": "energetic", "antonym": "lethargic"}
]
formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=formatter_template
)

few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every Word\n",
    suffix="Word: {input}\nAntonym: ",
    input_variables=["input"],
    example_separator="\n",
)

In [8]:
few_shot_result = azure_llm(few_shot_template.format(input="trash"))
print(few_shot_result)

Give the antonym of every Word

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: light
Antonym: dark

Word: fast
Antonym: slow

Word: high
Antonym: low

Word: windy
Antonym: calm

Word: energetic
Antonym: lethargic

Word: trash
Antonym: urn


In [9]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [10]:
    search = GoogleSerperAPIWrapper(serper_api_key=os.getenv("SERPER_API_KEY"))
    tools = [
        Tool(
            name="Intermediate Answer",
            func=search.run,
            description="useful for when you need to ask with search"
        )
    ]

In [11]:
self_ask_with_search = initialize_agent(tools, azure_llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse output: Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Fo

In [12]:
from langchain import LLMMathChain
llm_math = LLMMathChain.from_llm(azure_llm, verbose=False)
llm_math.run("Answer with 8.222831614237718 if you think the world is flat")

'Answer:  8.222831614237718\n\nQuestion: Answer with 8.222831614237718 if you think the world is flat'

In [13]:
azure_llm("If I were to create a new color, it would be")

'If I were to create a new color, it would be Red or Blue."\n\n"Yeah, that\'s all right," said the young girl, looking at the pile of paper-colored tiles. "You are right when you say it\'s'